# Computing Møller-Plesset 2nd order corrections

## Overview

This demo notebook relates to the following [Qiskit Nature](https://qiskit.org/documentation/nature/apidocs/qiskit_nature.html) pull requests.

* Migrate MP2Info [#590](https://github.com/Qiskit/qiskit-nature/pull/590)
* Add HF initial point [#632](https://github.com/Qiskit/qiskit-nature/pull/632)

In addition to adding the option of using a Møller-Plesset-informed initial point for [VQE](https://qiskit.org/documentation/stubs/qiskit.algorithms.VQE.html) with a [UCC](https://qiskit.org/documentation/nature/stubs/qiskit_nature.circuit.library.UCC.html)+[HF](https://qiskit.org/documentation/nature/stubs/qiskit_nature.circuit.library.HartreeFock.html) ansatz, these PRs address Issue [#571](https://github.com/Qiskit/qiskit-nature/issues/571) by introducing [explicit initial points](https://qiskit.org/documentation/nature/apidocs/qiskit_nature.algorithms.initial_points.html#module-qiskit_nature.algorithms.initial_points) in [VQEUCCFactory](https://qiskit.org/documentation/nature/stubs/qiskit_nature.algorithms.VQEUCCFactory.html) and [VQEUVCCFactory](https://qiskit.org/documentation/nature/stubs/qiskit_nature.algorithms.VQEUVCCFactory.html).

## Set-up

This demo was tested using Qiskit Nature `0.4.1`. The release notes for `0.4.0` can be found [here](https://qiskit.org/documentation/nature/release_notes.html#release-notes-0-4-0).

The following shell code should provide a suitable environment to run this demo.

```sh
    python3.9 -m venv .venv
    source .venv/bin/activate
    pip install qiskit-nature==0.4.1
    pip install qiskit-aer==0.10.4
    pip install pyscf==2.0.1
    pip install ipykernel==6.15.0
```

## Explicit initial points

Previously, the VQE+U(V)CC factories implicitly set the initial point within Qiskit Terra's VQE algorithm. This led to the problems raised in Issue [#571](https://github.com/Qiskit/qiskit-nature/issues/571); primarily, obfuscation of how and where the initial point was selected.

To resolve this, we have made explicit classes to compute the relevant initial point for a given ansatz. For example, to use VQE UCC with a Hartree-Fock initial point.

```python
    hf_initial_point = HFInitialPoint()
    vqe_ucc_factory = VQEUCCFactory(quantum_instance, initial_point=hf_initial_point)
```

This is also the default behavior when `initial_point=None`.

They can also be used outside of the factories by passing the result directly to the VQE.

```python
    hf_initial_point = HFInitialPoint()
    hf_initial_point.ansatz = your_ucc_ansatz
    initial_point = hf_initial_point.to_numpy_array()
```

We'll see a full example of this below.

## Møller-Plesset 2nd order corrections

Qiskit Aqua featured a utility class [MP2Info](https://github.com/Qiskit/qiskit-aqua/blob/main/qiskit/chemistry/mp2info.py) that computed the [Møller-Plesset 2nd order corrections](https://en.wikipedia.org/wiki/M%C3%B8ller%E2%80%93Plesset_perturbation_theory). This is now re-introduced in Qiskit Nature using an explicit initial point class.

Møller–Plesset perturbation theory (MP) is a method in computational chemistry that improves on the Hartree–Fock method by adding electron correlation effects by means of Rayleigh–Schrödinger perturbation theory (RS-PT). Currently, we have introduced these corrections to second-order (MP2).

Each double excitation—indexed by $i,a,j,b$, where $i,j(a,b)$ are the initial(final) orbital indices—has a correction coefficient,

$$
    c_{i,a,j,b} = -\frac{2 T_{i,a,j,b} - T_{i,b,j,a}}{E_b + E_a - E_i - E_j},
$$

where $E$ is the orbital energy and $T$ is the molecular orbital integral matrix. See the learning resources below for the derivation of these corrections.

The double-excitation coefficients are intended to be used as initial point values for the corresponding operators in VQE when using a UCC ansatz. This should introduce an initial point that is closer to the ground state point, leading to fewer evaluations.


## Using the MP2InitialPoint with a VQE+UCC factory

In [1]:
from qiskit import Aer
from qiskit.utils import QuantumInstance

from qiskit_nature.algorithms import VQEUCCFactory
from qiskit_nature.algorithms.initial_points.mp2_initial_point import MP2InitialPoint
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.algorithms import GroundStateEigensolver

from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer

molecule = Molecule(
    geometry=[["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.735]]], charge=0, multiplicity=1
)

driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)

freeze_core = FreezeCoreTransformer()

problem = ElectronicStructureProblem(driver, transformers=[freeze_core])

qubit_converter = QubitConverter(JordanWignerMapper())

initial_point = MP2InitialPoint()

quantum_instance = QuantumInstance(backend=Aer.get_backend("aer_simulator_statevector"))
vqe_ucc_factory = VQEUCCFactory(quantum_instance, initial_point=initial_point)

vqe_solver = GroundStateEigensolver(qubit_converter, vqe_ucc_factory)
res = vqe_solver.solve(problem)

print(res)

/var/folders/9v/kgdwv_rn2ng01vpgb16tvfk00000gn/T/ipykernel_19336/225945959.py:35: DeprecationWarning: VQEUCCFactory Constructor: quantum_instance is no longer a positional argument as of version 0.4 and will be removed no sooner than 3 months after the release. Instead use it as a keyword argument
  vqe_ucc_factory = VQEUCCFactory(quantum_instance, initial_point=initial_point)


=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275022959
  - computed part:      -1.857275022959
  - FreezeCoreTransformer extracted energy part: 0.0
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.13730602851
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.38894856]
    - computed part:      [0.0  0.0  1.38894856]
    - FreezeCoreTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  0.00000014]  Total: 0.00000014
                 (debye): [0.0  0.0  0.00000036]  Total: 0.00000036
 


## Using the MP2InitialPoint outside of a factory

In [2]:
from qiskit.algorithms.optimizers import L_BFGS_B
from qiskit.algorithms import VQE

from qiskit_nature.circuit.library import UCCSD
from qiskit_nature.settings import settings
from qiskit_nature.drivers import Molecule


settings.dict_aux_operators = True

molecule = Molecule(
    geometry=[["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.735]]],
    charge=0,
    multiplicity=1,
)

driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)

problem = ElectronicStructureProblem(driver)

# generate the second-quantized operators
second_q_ops = problem.second_q_ops()
main_op = second_q_ops["ElectronicEnergy"]

driver_result = problem.grouped_property_transformed

particle_number = driver_result.get_property("ParticleNumber")
electronic_energy = driver_result.get_property("ElectronicEnergy")

num_particles = (particle_number.num_alpha, particle_number.num_beta)
num_spin_orbitals = particle_number.num_spin_orbitals

# setup the classical optimizer for VQE
optimizer = L_BFGS_B()

# setup the qubit converter
converter = QubitConverter(JordanWignerMapper())

# map to qubit operators
qubit_op = converter.convert(main_op, num_particles=num_particles)

# setup the ansatz for VQE
ansatz = UCCSD(
    qubit_converter=converter,
    num_particles=num_particles,
    num_spin_orbitals=num_spin_orbitals,
)

mp2_initial_point = MP2InitialPoint()
mp2_initial_point.grouped_property = driver_result
mp2_initial_point.ansatz = ansatz
initial_point = mp2_initial_point.to_numpy_array()

# set the backend for the quantum computation
backend = Aer.get_backend("aer_simulator_statevector")

# setup and run VQE
algorithm = VQE(
    ansatz, optimizer=optimizer, quantum_instance=backend, initial_point=initial_point
)

result = algorithm.compute_minimum_eigenvalue(qubit_op)
print(result.eigenvalue.real)

electronic_structure_result = problem.interpret(result)
print(electronic_structure_result)


-1.94e-16
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): 0.0
  - computed part:      0.0
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): 0.719968994449
 
=== MEASURED OBSERVABLES ===
 
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 


## Conclusion

We've given an overview of some recent changes in Qiskit Nature that were introduced in Pull Requests [#590](https://github.com/Qiskit/qiskit-nature/pull/590) and [#632](https://github.com/Qiskit/qiskit-nature/pull/632). These changes were included in the `0.4.0` [release](https://qiskit.org/documentation/nature/release_notes.html#release-notes-0-4-0) of Qiskit Nature.

#632 adds Hartree-Fock (HF) and vibrational self-consistent field (VSCF) initial point classes. `HFInitialPoint` and `VSCFInitialPoint` are to be used with `UCC`, and `UVCC` ansatzes, respectively.

This follows the introduction of `MP2InitialPoint` in #590 for computing the initial point using the Møller-Plesset 2nd Order (MP2) corrections. `MP2InitialPoint`, `HFInitialPoint`, and `VSCFInitialPoint` all inherit from the abstract base class `InitialPoint`. These initial points are intended to be used as explicitly-declared starting VQE parameters when using a UCC ansatz. 

The corresponding double-excitation coefficients from the MP2 corrections are intended to introduce an initial point that is closer to the ground state point, leading to fewer overall evaluations for VQE when compared to the HF initial point.

## Learning resources

Qiskit Textbook (Outdated: uses Aqua library)
* [Simulating Molecules using VQE](https://learn.qiskit.org/course/ch-applications/simulating-molecules-using-vqe)

Qiskit VQE tutorials (Outdated: uses Aqua library)
* [Advanced VQE usage](https://qiskit.org/documentation/tutorials/algorithms/04_vqe_advanced.html)
* [Monitoring VQE convergence](https://qiskit.org/documentation/tutorials/algorithms/02_vqe_convergence.html)
* [VQE on Aer simulator with noise](https://qiskit.org/documentation/tutorials/algorithms/03_vqe_simulation_with_noise.html)

Qiskit Nature tutorials
* [Ground state solvers](https://qiskit.org/documentation/nature/tutorials/03_ground_state_solvers.html)
* [Excited states solvers](https://qiskit.org/documentation/nature/tutorials/04_excited_states_solvers.html)

Derivation of the Møller-Plesset 2nd order corrections:
* [Christopher J. Cramer, Essentials of Computational Chemistry (Second Edition), Chapter 7](https://www.researchgate.net/file.PostFileLoader.html?id=57838076dc332deafc078da6&assetKey=AS%3A382623223435264%401468235894332)

A full tutorial on using VQE with the new explicit initial point classes is planned. This will also demonstrate the potential advantage of fewer evaluations to reach the ground state when using an MP2 initial point.

---
Presented: [Qiskit DemoDay 2022-06-23](https://github.com/qiskit-community/feedback/wiki/Qiskit-DemoDays#jun-23-2022)

Author: Declan Millar, PhD <<declan.millar@ibm.com>>